In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
import pymysql
pymysql.install_as_MySQLdb()
from urllib.parse import quote_plus as urlquote
from sqlalchemy.engine import create_engine# Create the sqlalchemy engine and connection


In [2]:
import json
with open('/Users/troniceray/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [3]:
!pip install yelpapi

In [4]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [5]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()


<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
# Open the jsonfile and display the first level of keys
# open API results WITH json module
import json
with open('/Users/troniceray/.secret/yelp_api.json') as f:   #adjust for your path
    json_file = json.load(f)
    
## What type is top-level of json?
type(json_file)


dict

In [7]:
# set our API call parameters 
LOCATION = 'San Antonio, TX'
TERM = 'sushi'

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_sushi_SA.json"
JSON_FILE


'Data/results_in_progress_sushi_SA.json'

In [9]:
import os, json, math, time

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_sushi_SA.json already exists.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [12]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [13]:
## How many results total?
total_results = results['total']
total_results


287

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


15

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [17]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [18]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8aUAFZSENBgdC17hPpKp5g,wild-goji-restaurant-and-bar-san-antonio-3,Wild Goji Restaurant & Bar,https://s3-media1.fl.yelpcdn.com/bphoto/KApU2Q...,False,https://www.yelp.com/biz/wild-goji-restaurant-...,1202,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 29.522726203712143, 'longitude': ...","[restaurant_reservation, delivery]",$$,"{'address1': '7115 Blanco Rd', 'address2': 'St...",+12102770638,(210) 277-0638,3237.383488
1,0KczvSbrDfq9lHm7ZtF2rQ,piranha-izakaya-san-antonio-3,Piranha Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/ql-JIa...,False,https://www.yelp.com/biz/piranha-izakaya-san-a...,484,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 29.49438, 'longitude': -98.479}","[restaurant_reservation, delivery]",$$,"{'address1': '260 E Basse Rd', 'address2': 'St...",+12108221088,(210) 822-1088,3439.453518
2,l7tVQWHt14p6fcGdUvx3xw,shiro-japanese-sushi-bistro-san-antonio,Shiro Japanese Sushi Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/MBJHgv...,False,https://www.yelp.com/biz/shiro-japanese-sushi-...,221,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 29.4361425988757, 'longitude': -9...",[delivery],$$$,"{'address1': '107 W Jones Ave', 'address2': ''...",,,7240.257501
3,XLAUQDLaKVt6zkoY85pCKg,toroko-sushi-san-antonio,Toro'Ko Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/oApJJY...,False,https://www.yelp.com/biz/toroko-sushi-san-anto...,19,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 29.57580186090349, 'longitude': -...","[restaurant_reservation, pickup, delivery]",NaN,"{'address1': '14510 NW Military Hwy', 'address...",+12102778004,(210) 277-8004,9507.000189
4,0IztAsYiVX5GRh39yW0fkw,sushishima-castle-hills-castle-hills-2,Sushishima - Castle Hills,https://s3-media3.fl.yelpcdn.com/bphoto/AkvtSt...,False,https://www.yelp.com/biz/sushishima-castle-hil...,236,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 29.535244301536043, 'longitude': ...",[pickup],$$,"{'address1': '2211 NW Military Hwy', 'address2...",+12108880363,(210) 888-0363,4614.772847


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
282,Q9taWRIDBq3FALQiht7kUw,the-cheesecake-factory-san-antonio-san-antonio-12,The Cheesecake Factory,https://s3-media2.fl.yelpcdn.com/bphoto/Q1KdLo...,False,https://www.yelp.com/biz/the-cheesecake-factor...,609,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.0,"{'latitude': 29.592382, 'longitude': -98.610858}",[],$$,"{'address1': '15900 La Cantera Pkwy', 'address...",+12105581900,(210) 558-1900,14341.266777
283,bcFjXrZZ_dbIoGZNVxmSXA,h-e-b-plus-san-antonio-14,H-E-B plus!,https://s3-media3.fl.yelpcdn.com/bphoto/1RW4hx...,False,https://www.yelp.com/biz/h-e-b-plus-san-antoni...,22,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 29.379990584309308, 'longitude': ...",[],$,"{'address1': '4100 S New Braunfels Ave', 'addr...",+12105313100,(210) 531-3100,13846.739121
284,-IvBKbHtdxN7s-1VovOxBg,h-e-b-san-antonio-63,H-E-B,https://s3-media2.fl.yelpcdn.com/bphoto/3MPGwR...,False,https://www.yelp.com/biz/h-e-b-san-antonio-63?...,76,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 29.5622973331857, 'longitude': -9...",[],$$,"{'address1': '12777 Ih 10 W', 'address2': '', ...",+12105583981,(210) 558-3981,10404.742043
285,thoppZoZ0yzdlz-8RexoFg,topgolf-san-antonio,Topgolf,https://s3-media1.fl.yelpcdn.com/bphoto/EaJhU0...,False,https://www.yelp.com/biz/topgolf-san-antonio?a...,501,"[{'alias': 'newamerican', 'title': 'New Americ...",3.5,"{'latitude': 29.59409054130535, 'longitude': -...",[delivery],$$,"{'address1': '5539 N Lp 1604 W', 'address2': N...",+12102022694,(210) 202-2694,13550.982887
286,zGljSEDub70vPTe0D7JWxw,mia-marcos-pizza-selma,Mia Marco's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/Og1WQw...,False,https://www.yelp.com/biz/mia-marcos-pizza-selm...,199,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.584521, 'longitude': -98.306215}",[delivery],$$,"{'address1': '15939 IH 35 N', 'address2': '', ...",+12106678441,(210) 667-8441,22493.871237


In [22]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_sushi_SA.csv.gz', compression='gzip',index=False)
